In [19]:
import torch
import torchvision
import torch.nn as nn
import numpy
import matplotlib.pyplot as plt

In [20]:
device = ('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [21]:
transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize((256,256)),
    torchvision.transforms.CenterCrop((227,227)),
    torchvision.transforms.ToTensor()
])

In [22]:
batch_size = 32
learning_rate = 1e-2
learning_momentum = 0.9
num_epoch = 15

In [23]:
train_data = torchvision.datasets.CIFAR100(root='/content/sample_data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_data, shuffle=True, batch_size=batch_size, num_workers=4)

test_data = torchvision.datasets.CIFAR100(root='/content/sample_data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_data, shuffle = False, batch_size = batch_size, num_workers = 4)

Files already downloaded and verified
Files already downloaded and verified


In [24]:
class AlexNet(nn.Module):
  def __init__(self):
    super().__init__()

    self.conv_net = nn.Sequential(
        nn.Conv2d(in_channels=3, out_channels=96, kernel_size=11, stride=4),
        nn.LocalResponseNorm(size=5, alpha=0.0001, beta=0.75, k=2),
        nn.MaxPool2d(kernel_size=3, stride =2),
        nn.ReLU(),

        nn.Conv2d(96, 256, 5, padding=2),
        nn.LocalResponseNorm(size=5, alpha=0.0001, beta=0.75, k=2),
        nn.MaxPool2d(3,stride=2),
        nn.ReLU(),

        nn.Conv2d(256, 384, 3, padding=1),
        nn.ReLU(),
        nn.Conv2d(384, 384, 3, padding=1),
        nn.ReLU(),

        nn.Conv2d(384,256, 3, padding=1),
        nn.MaxPool2d(3, stride=2),
        nn.ReLU(),
    )

    self.fc_net = nn.Sequential(
        nn.Dropout(p=0.5),
        nn.Linear(256*6*6, 4096),
        nn.ReLU(),
        nn.Dropout(p=0.5),
        nn.Linear(4096, 4096),
        nn.ReLU(),
        nn.Linear(4096, 100)
    )

  def forward(self,x):
    x = self.conv_net(x)
    x = x.view(-1,256*6*6)
    x = self.fc_net(x)
    return x

In [25]:
model = AlexNet()
model.to(device)
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr= learning_rate, momentum=learning_momentum)

In [26]:
print(model)

AlexNet(
  (conv_net): Sequential(
    (0): Conv2d(3, 96, kernel_size=(11, 11), stride=(4, 4))
    (1): LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=2)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): ReLU()
    (4): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (5): LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=2)
    (6): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): ReLU()
    (8): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU()
    (10): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU()
    (12): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): ReLU()
  )
  (fc_net): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
    (2): R

In [ ]:
torch.autograd.set_detect_anomaly(True)
for epoch in range(num_epoch):
  for i, (images, labels) in enumerate(train_loader):
    images = images.to(device)
    labels = labels.to(device)

    prediction = model(images)
    loss = loss_func(prediction, labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (i+1)%1000 == 0:
      print(f"[{epoch, i+1}], loss : {loss}")


In [ ]:
total = correct = 0
with torch.no_grad():
  for data in test_loader:
    images, labels = data
    images = images.to(device)
    labels = labels.to(device)

    output = model(images)
    _, predicted = torch.max(output, 1)
    total += labels.size(0)
    correct += (predicted==labels).sum().item()

  print(f"The Accuracy is {100*(correct/total)}%")